In [ ]:
!pip install openai-agents
!pip install 'openai-agents[litellm]'
# !pip install -Uq openai-agents "openai-agents[litellm]"

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

In [ ]:
from __future__ import annotations

import asyncio

from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
# from google.colab import userdata

set_tracing_disabled(disabled=True)

#       Company / LLM Model
MODEL = "gemini/gemini-2.0-flash"
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")




# Using Main Function

In [ ]:
def main(model: str, api_key: str):
    agent = Agent(
        name="Assistant",
        instructions="You only respond in three lines.",
        model=LitellmModel(model=model, api_key=api_key),

    )

    result = Runner.run_sync(agent, "What's AI Agent?")
    print(result.final_output)

main(model=MODEL, api_key=GEMINI_API_KEY)


AI Agents are intelligent systems.
They perceive environments and take actions.
Autonomously achieving specific goals.



# Direct Method

In [ ]:
agent = Agent(
        name="Assistant",
        instructions="You only respond in three lines.",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),

    )

result = Runner.run_sync(agent, "What's AI Agent?")
print(result.final_output)


An AI agent is a software entity.
It perceives its environment and acts autonomously.
To achieve specific goals in that environment.



# Adding Custom Function/Tool

You can also run this without main() function.

In [ ]:
@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."


async def main(model: str, api_key: str):
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=LitellmModel(model=model, api_key=api_key),
        tools=[get_weather],
    )

    result = await Runner.run(agent, "What's the weather in Lahore?")
    print(result.final_output)

asyncio.run(main(model=MODEL, api_key=GEMINI_API_KEY))

I need the city.
Please provide it, I'll check it,
Then give weather back.



# Handsoff

In [ ]:
@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."

weather_agent = Agent(
        name="WeatherAssistant",
        instructions="You will share weather.",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
        tools=[get_weather],
        handoff_description="Weather Assistant is expert in Weather Questions"
    )

panacloud_agent = Agent(
        name="PanacloudAssistant",
        instructions="You will answer Panacloud related queries.",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
        handoff_description="Panacloud Assistant is specialized for all Panacloud Queries. IT can search for Panacloud and provide answers."
    )

# Triage Agent = Boos / Crew Leader
agent = Agent(
        name="GeneralAssistant",
        instructions="You Chat with user for weather related queries and handoffs to WeatherAssistant or for Panacloud related queries handoffs to Panacloud Assistant.",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
        handoffs = [weather_agent, panacloud_agent],
    )

result = await Runner.run(agent, "Hi")
print(result.final_output)

Hi there! How can I help you today?



In [ ]:
dir(result)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_last_agent',
 'context_wrapper',
 'final_output',
 'final_output_as',
 'input',
 'input_guardrail_results',
 'last_agent',
 'last_response_id',
 'new_items',
 'output_guardrail_results',
 'raw_responses',
 'to_input_list']

In [ ]:
result.last_agent.name

'GeneralAssistant'

In [ ]:
result = await Runner.run(agent, "What's the weather in Lahore?")
print(result.final_output)

[debug] getting weather for Lahore
The weather in Lahore is sunny.



In [ ]:
result.last_agent.name

'WeatherAssistant'

In [ ]:
result = await Runner.run(agent, "What's Panacloud stands for?")
print(result.final_output)

PIAIC stands for **Presidential Initiative for Artificial Intelligence & Computing**.



In [ ]:
result.last_agent.name

'PiaicAssistant'

In [ ]:
result = await Runner.run(agent, "In Which country or city Panacloud located and whats the weather there?")
print(result.final_output)

PIAIC (Presidential Initiative for Artificial Intelligence and Computing) is located in **Pakistan**, primarily in the city of **Karachi**. However, it has initiatives and centers in other cities of Pakistan as well.

Regarding the weather in Karachi: As of October 26, 2023, the weather in Karachi is generally sunny and warm. Temperatures can range from the mid-20s to the low 30s degrees Celsius (around 75-90 degrees Fahrenheit) during the day. The humidity is also typically moderate. Of course, weather can vary, so it's always a good idea to check a local weather forecast for the most up-to-date information.



In [ ]:
result.last_agent.name

'PiaicAssistant'

In [ ]:
@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."

weather_agent = Agent(
        name="WeatherAssistant",
        instructions="You will share weather.",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
        tools=[get_weather],
        handoff_description="Weather Assistant is expert in Weather Questions"
    )

panacloud_agent = Agent(
        name="PanacloudAssistant",
        instructions="You will answer Panacloud related queries & for weather related queries pass it to Weather Assistant",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
        handoff_description="Panacloud Assistant is specialized for all Panacloud Queries. IT can search for Panacloud and provide answers.",
        handoffs=[weather_agent],
    )

# Triage Agent = Boos / Crew Leader
agent = Agent(
        name="GeneralAssistant",
        instructions="You Chat with user for weather related queries and handoffs to WeatherAssistant or for Panacloud related queries handoffs to Panacloud Assistant.",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
        handoffs = [weather_agent, panacloud_agent],
    )

result = await Runner.run(agent, "Hi")
print(result.final_output)

Hi, how can I help you today?



In [ ]:
result = await Runner.run(agent, "In Which country or city Panacloud located and whats the weather there?")
print(result.final_output)
print(result.last_agent.name)

PIAIC is located in Pakistan, but I can't tell you the weather there. I'll transfer you to the Weather Assistant for that.

PiaicAssistant


In [ ]:
@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."

weather_agent = Agent(
        name="WeatherAssistant",
        instructions="You will share weather.",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
        tools=[get_weather],
        handoff_description="Weather Assistant is expert in Weather Questions"
    )

panacloud_agent = Agent(
        name="PanacloudAssistant",
        instructions="You will answer Panacloud related queries & for weather related queries pass it to Weather Assistant",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
        handoff_description="Panacloud Assistant is specialized for all Panacloud Queries. IT can search for Panacloud and provide answers.",
        handoffs=[weather_agent],
    )

# Triage Agent = Boos / Crew Leader
agent = Agent(
        name="GeneralAssistant",
        instructions="You Chat with user for weather related queries and handoffs to WeatherAssistant or for Panacloud related queries handoffs to Panacloud Assistant.",
        model=LitellmModel(model=MODEL, api_key=GEMINI_API_KEY),
        tools = [weather_agent.as_tool(tool_name="Weather_tool", tool_description="You will share weather.") ,
                 panacloud_agent.as_tool(tool_name="Panacloud_tool", tool_description="Panacloud Assistant is specialized for all Panacloud Queries. IT can search for Panacloud and provide answers.")],
    )

result = await Runner.run(agent, "What is Panacloud and what is weather in Lahore?")
print(result.final_output)

[debug] getting weather for Lahore
PIAIC is Pakistan Information and Technology Institute. The weather in Lahore is sunny. Do you have any other questions?

